# 乗算回路
汎用計算機なので掛け算もできます。今回は基本的な回路で２進数の掛け算を見てみましょう。

## ２進数の掛け算について
考えてみましたが、シンプルです。２つの数をくらいごとにかけてずらして足し合わせます。その際にキャリービットとして桁上がりを考慮します。

0*0=0
0*1=0
1*0=0
1*1=1

つまり、11のときに1となる回路を作れば良さそうです。これはccxです。あとは各くらいを足し合わせれば大丈夫でしょう。



## 例題
習うより慣れろでいきます。まずは、1*2=?をときます。2は2進数で10ですので、01*10=0010。つまり1*2=2となります。



```
    01
*  10
-------
    00
 01
-------
    0
 0
-------
 0010
  ```

では、早速実装へ。まずは2進数の数を２つ用意します。a*bを考えますが、aの0のくらいとaの2のくらいを用意して、それぞれa0とa2とします。bも同様です。

今回最終的に実現するのは|a,b,x> => |a,b,a*b>ととりあえず目標を置いてみます。求めたいのはx0,x2,x4,x8です。cは途中の計算用のビット。zは桁上がりビットを想定します。

<img src="./img/010_basic_multi01.png">


In [89]:
from blueqat import Circuit

C1 = Circuit().ccx[0,1,2].ccx[1,3,5].ccx[0,4,6].ccx[3,4,7].ccx[5,6,8].ccx[7,8,9].cx[2,10].cx[5,11].cx[6,11].cx[7,12].cx[8,12].cx[9,13] 

#00 * 00 = 0000
(Circuit() + C1).m[:].run(shots=100)

Counter({'00000000000000': 100})

In [81]:
#01 * 01 = 0001
(Circuit().x[0].x[1] + C1).m[:].run(shots=100)

Counter({'11100000001000': 100})

In [82]:
#10 * 01 = 0010
(Circuit().x[3].x[1] + C1).m[:].run(shots=100)

Counter({'01010100000100': 100})

In [83]:
#01 * 10 = 0010
(Circuit().x[0].x[4] + C1).m[:].run(shots=100)

Counter({'10001010000100': 100})

In [84]:
#10 * 10 = 0100
(Circuit().x[3].x[4] + C1).m[:].run(shots=100) 

Counter({'00011001000010': 100})

In [86]:
#11 * 10 = 0110
(Circuit().x[0].x[3].x[4] + C1).m[:].run(shots=100)

Counter({'10011011000110': 100})

In [87]:
#10 * 11 = 0110
(Circuit().x[1].x[3].x[4] + C1).m[:].run(shots=100)

Counter({'01011101000110': 100})

In [88]:
#11 * 11 = 1001
(Circuit().x[0].x[1].x[3].x[4] + C1).m[:].run(shots=100) 

Counter({'11111111111001': 100})

どうでしょうか、すべての足し算の場合が4通り出てきました。条件を満たす計算が重ね合わせを使って実現できました。上記は、

0+0=00
0+1=10
1+0=10
1+1=01

となっています（桁がみづらいですが）。以上で重ね合わせと加算回路を使った計算ができました。